In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

%matplotlib inline

In [2]:
# read file
text = pd.read_table('./sample_movie_review.txt', delimiter='\t', quoting=3)

In [3]:
# defind the head of the html file
start_string = """
<!DOCTYPE html>
<html>
<head>
<style>@import "textstyle.css"</style>
</head>
<body>
<h1 align="middle">Movie Review Visualization</h1> 
<h2> 1. Top Positive and Negative Words</h2>
<center><img src="./tmp.png" width=50% align="middle"></center>
<h2> 2. Detailed Text Analysis</h2>
"""

In [4]:
# define positive and negative words
positive = ["cool", "like", "talented", "great", "appreciate"]
negative = ["odd", "bad", "stupid", "shortcoming"]

In [5]:
# process and write to html file
with open('./sample_visualization.html', 'w') as f:
    f.write(start_string)
    f.write('<ul>\n')
    
    for num in range(len(text)):
        idx = text['id'][num][1:-1]
        sentiment = text['sentiment'][num]
        
        # split and replace the review words
        tmp_text = text['review'][num][1:-1]
        tmp_text = tmp_text.replace('\\', '')
        tmp_text = BeautifulSoup(tmp_text, 'lxml').get_text()
        # add space after comma that has no space
        tmp_text = re.sub(r'([,]+)(?![ ])', r'\1 ', tmp_text)
        tmp_text = tmp_text.split(' ')
        
        # write the header part of each review
        f.write('<hr>\n&ensp;')
        f.write('<strong> id: </strong> ' + str(idx) + ' &emsp; &emsp;\n')
        f.write('<strong> sentiment: </strong>' + str(sentiment) + ' <br>\n')
        f.write('<hr>\n')
        
        # find all the indexes for the positive and negative words
        pos_idx = []
        neg_idx = []
        for word in positive:
            # regular pattern match
            regex = re.compile('.*(' + word + ').*', re.IGNORECASE)
            pos_idx += [i for i in range(len(tmp_text)) if regex.search(tmp_text[i])]
        
        for word in negative:
            # regular pattern match
            regex = re.compile('.*(' + word + ').*', re.IGNORECASE)
            neg_idx += [i for i in range(len(tmp_text)) if regex.search(tmp_text[i])]
            
        # replace all positive words with pre-defined html class
        for i in pos_idx:
            tmp_text[i] = '<span class="pos">' + tmp_text[i] + '</span>'
            
        # replace all negative words with pre-defined html class
        for i in neg_idx:
            tmp_text[i] = '<span class="neg">' + tmp_text[i] + '</span>'
            
        # write into html file
        f.write(' '.join(tmp_text))  
        f.write('\n<br><br>\n\n')
        
    # end of the html file
    f.write('</ul>\n</body>\n</html>\n')